# Difusión sobre el Product Space 

En este trabajo implementaremos un modelo de de difusión sobre el Product Space, haciendo uso del formalismo EB-DEVS. Utilizaremos los mismos datos que en los trabajos originales (extraidos de Harvard Dataverse) y exploraremos qué ocurre con distintas dificultades de difusión (valores de $\Omega$) y simularemos con criterio de retroalimentación global y sin éste.

### Introducción al modelo Product Space

Diremos que un país $c$ exporta un producto $p$ cuando $RCA_{c,p} = \frac{\$ \text{ de p exportados por c }}{\$ \text{ promedio de exportaciones de p}} > 1$.

Para simplificar la notación $x_p^c = 1 \text{ si } RCA_{c,p} > 1$ (y 0 sino). 

Luego se define la similitud entre productos de la siguiente manera:

$$ \varphi_{i,j}=\min\{P(RCA_i|RCA_j), P(RCA_j|RCA_i)\} = \sum_c x_i^c x^c_j \times\min\{\frac{1}{\sum_c x_i^c},\frac{1}{\sum_c x_j^c}\} $$ 

con $ P(RCA_i,RCA_j) = \frac{\sum_c x_i^c x_j^c}{\#países} $.

Notar que la matrix que resulta $\Phi = \{\varphi_{i, j}\}_{i,j}$ depende de $x$ de todos los paises y productos.

### Introducción de variable temporal y progreso de la difusión

Si bien el Product Space observa versiones estáticas de las variables mencionadas arribas, nosotros buscamos agregarle tiempo y progreso. La dimensión temporal resulta natural argegarla a $x_{p,t}^c$ para todo $p$ y $c$. Esto a su vez deriva las versiones temporales de $\varphi_{i,j}^t$ y $\Phi^t$.

Para el concepto de progreso establecemos ciertas reglas:

- Sea $\Omega$ el umbral de complejidad que se debe cruzar para desarrollar un producto.
- Si $x^c_{p,t}=0$ entonces la única manera de que el producto sea exportado es si $\Omega < \max_{i / x_{i,t} = 1}{\varphi_{p,i}}$. Resultando en $x^c_{p,t+1}=1$.
- Si para $t$ $x_{p,t}^c = 1$ entonces $\forall t' > t \; x_{p,t}^c = 1$. O sea, una vez exportado un bien por un país, este siempre lo exporta por el resto de la simulación.


#### Simplificaciones cuentísticas

- Llamaremos $\overrightarrow{d}^c_p$ al vector que representa las distancias entre un pais $c$ y un producto $p$, cuya definición es $\overrightarrow{d}^c_p= \max_{j/ x_{j}^c =1} {\varphi_{p, j}}$.
- O de forma matricial $\overrightarrow{d}^c = \max fila(\Phi X^c)$ con $$ X^c = \begin{bmatrix}
x_1^c & 0 & \dots & 0 \\
0 & x_2^c & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & x_n^c
\end{bmatrix} $$

Notar que $\overrightarrow{d}^{c,t}_p$ también depende del tiempo.


### Variantes: retroalimentación vs local

Como mencionamos, simularemos dos variantes distintas, una con retroalimentación de los países y sus descubrimientos al resto del mundo y otra en la que el desarrollo se da de forma aislada en cada país y la complejidad de producir no cambia en función del tiempo.

- Variante global (o con retroalimentación): se basa principalmente en actualizar $\Phi^{t+1}$ con los datos de $x_{p,t}^c$ y luego que $\overrightarrow{d}^{c,t+1}_p$ utilice $\Phi^{t+1}$.
    
    Esta es la versión más sensata desde el punto de vista de la realidad, ya que suponemos que los avances y desarrollos que se dan en el mundo globalizado en que vivimos afectan la complejidad de producir. Pero como no removemos productos de las canastas de exportaciones, equivaldría a creer que la complejidad expresada en $\Phi$ siempre se reduce, cosa que por crisis, faltantes, perdidas de competitividad, etc. es un poco naïve.


- Variante local (o sin retroalimentación): se basa en hacer que $\Phi$ no dependa del tiempo, solamente se tiene en cuenta la $\Phi$ inicial, o sean las relaciones entre los paises y sus exportaciones en el instante inicial. Luego cada país progresa mediante la actualización de $X^c_t$. De esta forma los avances son aislados entre los paises y productos.

En el trabajo intentaremos ver si estos métodos tienen diferencias y en caso de que las tuvieran por qué surgen.


## Modelo Conceptual

Contamos con dos tipos de modelos atómicos:

- Los paises, que tienen noción de sus exportaciones competitivas y realizan el proceso de difusión local.
- El generador de eventos, que una vez por unidad de tiempo manda un mensaje a cada país para que difunda. En este mensaje, informa a los países del valor de $\Omega$, que toma del acoplado.

Ambos atómicos se sitúan en el acoplado de Espacio de Productos, que tiene conocimiento de la matriz $\Phi$ (macro estado). La $\Phi$ se actualizará con los cambios en el micro estado de los países si se usa el modelo con retroalimentación.

![Modelo Conceptual Product Space](Modelo_Conceptual_Product_Space.drawio.png "Modelo conceptual")

## Simulación

Implementamos el modelo usando EB-DEVS, lo que permite que el acomplado de Espacio de Productos tenga el macro estado de la simulación, y que este sea accesible por los atómicos para afectar su funcionamiento. Además, en el caso con retroalimentacón, permite que el acoplado actualice su estado con los cambios en el micro estado de los atómicos.

### Generacion de datos

Utilizamos los datos de comercio internacional [publicados por el _Growth Lab_ de la Universidad de Harvard](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/H8SFD2), en formato SITC Rev. 2 con códigos de producto de cuatro dígitos.

De este dataset, tomamos el RCA (que viene precalculado) para cada producto en un país y año, y a partir de estos datos generamos las matrices $X$ de cada año.

### Cómo simular

Para correr la simulación, se requiere Python3 con EB-DEVS y numpy instalado. Se puede correr el script `main.py` con los siguientes argumentos:

```
--duration, -d: cantidad de interaciones de difusión a realizar
--big-omega, -O: valor del parámetro de difusión (por defecto, 0.55)
--metrics-folder, -m: carpeta o directorio en el que guardar las métricas
--phi-matrix-update, -u: booleano que define si se utiliza el modelo con retroalimentación (por defecto, falso)
--X-matrices-file, -f: datos de entrada, en formato pkl, que contiene las matrices X (por defecto, data/stage1_data.pkl)
--year, -y: año de origen de la simulación, define con qué matriz X se empieza (por defecto, 2000)
```

El simulador genera dos archivos de salida con métricas:

- `omega.csv`: máximo, mínimo y promedio del vector $\overrightarrow{d}^{c, t}$ de un país en cada generación/iteración
- `exports.csv`: cantidad de productos exportados por un país en cada generación/iteración


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

networkx MST progresion para argentina global vs local

# Diferencias de exportaciones por omega y criterio de simulacion`

In [ ]:
## CARGA DE DATOS PARA MEDIR DIFERENCIAS ENRTRE CRITERIOS
exports_diff_dfs = []
exports_global_dfs = []
exports_local_dfs = []
for omega in ("0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.8", "0.9", "0.55", "0.65"):
    exports_global_df = pd.read_csv(f"experimentacion/metrics-true-{omega}/exports.csv")
    exports_global_df["omega"] = float(omega)
    exports_global_dfs.append(exports_global_df)

    exports_local_df = pd.read_csv(f"experimentacion/metrics-false-{omega}/exports.csv")
    exports_local_df["omega"] = float(omega)
    exports_local_dfs.append(exports_local_df)

    exports_diff_df = exports_global_df.set_index(["country", "generation"]) - exports_local_df.set_index(["country", "generation"])
    exports_diff_df["omega"] = float(omega)

    exports_diff_dfs.append(exports_diff_df)

exports_diff_df = pd.concat(exports_diff_dfs)
exports_diff_df.sort_values("omega", inplace=True)

exports_global_df = pd.concat(exports_global_dfs)
exports_global_df.sort_values("omega", inplace=True)

exports_local_df = pd.concat(exports_local_dfs)
exports_local_df.sort_values("omega", inplace=True)

Veamos primero cual es la tendencia de diferencias en exportaciones actualizando $\Phi$ en cada generacion (llamado criterio de simulacion global) versus no realizar esta actualizacion (llamado criterio de simulacion local).

En la bibliografia se menciona que esta actualizacion no trae beneficios claros. Tener en cuenta que actualizar $\Phi$ corresponde a volver a categorizar los productos.

Para esto elegimos visualizar la el avance de la diferencia absoluta segun cada umbral $\Omega$ para distintos paises. A medida avanza la simulacion, dado que es un proceso de difusion, se aproximan asintotas sobre las metricas observadas.

In [ ]:
paises = ["ARG", "BRA", "CHL", "AUS"]
# graficar las dos curvas
# TODO: fijar los paises
paises = set(paises + [country for country, _ in np.random.choice(exports_diff_df.index, 5 - len(paises))])
for pais in sorted(paises):
    df = exports_diff_df.loc[pais].reset_index().sort_values(["generation","omega"])
    fig = px.line(df,
                x="generation",
                y="count_exports",
                color="omega",
                title=f"Diferencia en cantidad de exportaciones para {pais} y omega segun criterio de simulacion",
                labels={"count_exports": "Diferencia de exportaciones", "generation": "Generacion", "country": "Pais"})
    fig.show()

Se observa que, para todos los paises, la diferencia se estabiliza en las ultimas generaciones (10ma generacion en adelante), por lo que tomaremos las comparativas con la menor divergencia (la ultima generacion disponible).

Veamos con mayor detalle como es la distribucion segun $\Omega$ (umbral de "descubrimiento").

In [ ]:
max_diff_df = (
    exports_diff_df
    .reset_index()
    .set_index(["generation", "country", "omega"])
    .loc[max(generation for _, generation in exports_diff_df.index)]
    .max(axis=1)
)
max_diff_df.rename("difference", inplace=True)
fig = px.box(
    max_diff_df.reset_index(),
    x="omega",
    y="difference",
    title=f"Distribucion de diferencias segun criterio de simulacion en cantidad de exportaciones x omega (ultima generacion)",
    labels={"difference": "Diferencia de exportaciones", "omega": "Omega"})
fig.show()

Claramente los mas interesantes de analizar son los valores de $\Omega = 0.5, 0.55, 0.6 $ y $ 0.65$. Esto se condice con la idea de que a valores muy pequenos de $\Omega$ el descubrimiento es facil y a valores muy elevados es muy dificil, por lo que las alternativas globales y locales pecan en los mismos aspectos ($\Omega$ seria mas significativo que actualizar $\Phi$).


#### Local vs global para $\Omega$ poco interesante

In [ ]:
# diferencia total en la cantidad de exportaciones de los omegas
UNINTRESTING_OMEGAS = (0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 0.9)
df_ = exports_diff_df[exports_diff_df.omega.isin(UNINTRESTING_OMEGAS)].reset_index().sort_values(["omega", "generation"])
fig = px.line(df_,
        color="country",
        x="generation",
        y="count_exports",
        facet_col="omega", facet_col_wrap=2,
        height=1000,
        width=2000,
        title=f"Evolucion de diferencias segun criterio de simulacion en cantidad de exportaciones (por omega)",
        labels={"count_exports": "Diferencia de exportaciones", "omega": "Omega", "generation": "Generation"})
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
fig.update_yaxes(matches=None, range=[0, df_.count_exports.max()])
fig.update_xaxes(matches=None)
fig.show()

Como vemos, no hay cambios de interes fuera de las primeras generaciones para ninguno de los omegas elegidos, por lo que para $\Omega = 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 0.9$ concluimos que las versiones locales y globales son indistinguibles.

#### Local vs global para $\Omega$ interesante

In [ ]:
INTRESTING_OMEGAS = (0.5, 0.55, 0.6, 0.65)
df_ = exports_diff_df[exports_diff_df.omega.isin(INTRESTING_OMEGAS)].reset_index().sort_values(["omega", "generation"])
fig = px.line(df_,
        color="country",
        x="generation",
        y="count_exports",
        facet_col="omega", facet_col_wrap=2,
        height=1000,
        width=2000,
        title=f"Evolucion de diferencias segun criterio de simulacion en cantidad de exportaciones (por omega)",
        labels={"count_exports": "Diferencia de exportaciones", "omega": "Omega", "generation": "Generation"})
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
fig.update_yaxes(matches=None, range=[0, df_.count_exports.max()])
fig.update_xaxes(matches=None)
fig.show()

Cabe notar que para $\Omega = 0.6$ y $0.65$ la el comportamiento entre la version local y la global no sufre grandes cambios, por lo que reclasificamos estos valores como de no interes y subsecuentemente como ambas versiones son indistinguibles para estos valores.

No obstante, $\Omega = 0.5$ y $0.55$ si muestran comportamientos significativamente distintos, por lo que no podemos concluir que la version local y la global tengan indistinguibles. Como lo que esta siendo graficado es global - local y la diferencia es positiva y mayor a 20 productos en la mayoria de los casos, interprestamos que la version global facilita la difusion.

Explicamos el comportamiento indistinguible de los casos clasificados como $\Omega$ poco interesante ya que para estos valores de $\Omega$ la difusion suele alcanzar maximos por ser muy facil difundir o directamente es muy dificil, entonces la retroalimentacion no aporta el suficiente peso como para contrarrestar los efectos negativos en el desarrollo de nuevos productos.

Por ultimo veamos la cantidad de productos exportados. 

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

paises

fig = make_subplots(len(paises), shared_xaxes=True, shared_yaxes=True)

for i, pais in enumerate(paises):
    global_df_ = exports_global_df[exports_global_df.country == pais]
    global_df_.sort_values(["omega", "generation"])
    omegas = set(global_df_.omega)
    for omega in omegas:
        df_ = global_df_[global_df_.omega==omega].sort_values("generation")
        fig.add_trace(
            go.Scatter(
                x=df_.generation,
                y=df_.count_exports,
                mode="lines",
                name=f"{pais} - {omega}",
            ),
            row=i+1, col=1,
        )

fig.show()

In [ ]:
exports_global_df

# Pruebas

In [ ]:
exports_global_df_copy = exports_global_df.copy()
exports_global_df_copy["local"] = False
exports_local_df_copy = exports_local_df.copy()
exports_local_df_copy["local"] = True
exports_df = exports_global_df_copy.append(exports_local_df)

In [ ]:
top_20 = exports_df[exports_df.generation == exports_df.generation.max()].nlargest(n=20, columns=["count_exports"]).country
exports_top_20_df = exports_df[exports_df.country.isin(top_20) | (exports_df.country == "ARG")]
fig = px.line(exports_top_20_df,
    x="generation",
    y="count_exports",
    color="country",
    title=f'Productos exportados por pais para los 20 exportadores mas diversificados + ARG (RCA > 1)',
    labels={"count_exports": "Exportaciones", "generation": "Generacion", "couintry": "Pais"},
)
fig.update_yaxes(range=[0, exports_df.count_exports.max()])
fig.show()